In [1]:
import tensorflow as tf
from tensorflow.keras import *
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [2]:
model=Sequential()
model.add(layers.Dense(2,activation='softmax'))
model.compile(optimizer=optimizers.Adam(),
             # loss=keras.losses.CategoricalCrossentropy(),  # 需要使用to_categorical
             loss=losses.binary_crossentropy,
              metrics=['accuracy'])

In [3]:
data_path='J:/VeReMi-Dataset/000_Select_data'
os.chdir(data_path)
datalist=os.listdir(data_path)
test=np.load('test.npy',allow_pickle=True)
val=np.load('val.npy',allow_pickle=True)
train=np.load('train.npy',allow_pickle=True)
train=train.T
val=val.T
test=test.T

In [13]:
def get_info(bucket_list,data_label_df):
    result=[]
    for i in data_label_df:
        result.append(i[bucket_list])
    return result

def regularize(raw_data):
    frame_list = []
    for df in raw_data:
        newDataFrame = pd.DataFrame(index=df.index)
        columns = df.columns.tolist()
        for c in columns:
            if c != 'type':
                d = df[c]
                try:
                    len(d[0])
                    tlist = []
                    for i in d:
                        tlist.append([])
                    for i in range(len(d[0])):
                        MAX = d[0][i]
                        for row in d:
                            if row[i] > MAX:
                                MAX = row[i]
                        MIN = d[0][i]
                        for row in d:
                            if row[i] < MIN:
                                MIN = row[i]
                        for j in range(len(d)):
                            if MAX == MIN:
                                tlist[j].append(1)
                            else:
                                tlist[j].append((d[j][i]-MIN)/ (MAX - MIN))
                    newDataFrame[c] = tlist
                except TypeError:
                    MAX = d.max()
                    MIN = d.min()
                    newDataFrame[c] = ((d - MIN) / (MAX - MIN)).tolist()
            else:
                newDataFrame[c] = df[c]
        frame_list.append(newDataFrame)
    return frame_list

In [19]:
def data_convert(data):#数据正则与空值填充
    data[1]=np.asarray(data[1]).astype('float64')
    data_regularized=regularize(data[0])
    for i in data_regularized:
        i.fillna(0.0,inplace=True)
    return data[1],data_regularized
trainY,trainX_regularized=data_convert(train)
valY,valX_regularized=data_convert(val)
testY,testX_regularized=data_convert(test)

ValueError: could not broadcast input array from shape (1475) into shape (183)

In [8]:
history = model.fit(trainX_regularized, trainY, batch_size=64, epochs=10, validation_data=(valX_regularized,valY))
effect=model.evaluate(testX_regularized,testY)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).